In [36]:
import os
import cv2
import shutil
# import deeplabcut
import numpy as np
import pandas as pd
import subprocess
import datetime 
import math
from joblib import Parallel, delayed
import glob

# import argparse
from matplotlib import pyplot as plt
from PIL import Image
import scipy.stats as stats
# from scipy.linalg import fractional_matrix_power

# import plotly.express as px
# from IPython.display import HTML

# from dash import Dash, dcc, html, Input, Output, no_update, callback
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
import base64
import random

In [8]:
# Extracting File names, titles and date created to extrach info from their DLC processing

project_name = 'DLC'
your_name = 'YR'

os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids')
h5files = []
vid_to_crop = []

today_1 = datetime.date(2023, 11, 14)
# today_1 = datetime.date.today()
today = str(today_1)
# today = "2023-10-29"
chkdate = today_1.strftime("%b%d")

proj_fold = project_name+'-'+your_name+'-'+today
# print(today)
target = os.path.join(os.getcwd(),proj_fold,'videos')
# print(target)
for file in os.listdir(target):
    if file.endswith("filtered.h5"):
        h5files.append(os.path.join(target,file))
for file in os.listdir(target):   
    if file.endswith("labeled.mp4"):
        vid_to_crop.append(os.path.join(target,file))
        
crop_source = []
if len(h5files)==len(vid_to_crop):
    for i in h5files:
        found = ''
        for j in vid_to_crop:
            x = os.path.splitext(i)[0].rstrip('_filtered.h5')
            y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
            if x==y:
                found = j
        k = os.path.basename(i)
        l = 'DLC' + k.split('DLC')[1] + 'DLC' + k.split('DLC')[2].rstrip('_filtered.h5')

        crop_source.append([i,found, l, chkdate])
c_s = np.array(crop_source)

In [9]:
for i in c_s:
    fl_pth, vid_pth, x, date = i

    cap = cv2.VideoCapture(vid_pth)

    tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(vid_pth, tot_frame)

/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vids_GCDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869912
/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampledDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869919
/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampled_72_2_5_5_AGC_claheDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869919


In [10]:
# Extracts confidence of each point in each video, grouped under points.

feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']#, 'RightEye_Inner', 'LeftEye_Inner']
# feature_names = ['RightEye_Pupil', 'LeftEye_Pupil', 'OutlineTop_Mid', 'NostrilsTop_Centre', 'RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']


conf = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    mul_cols = []
    for j in c_s:
        fl_pth, vid_pth, x, date = j
        name_check = os.path.basename(vid_pth).split('DLC')[0]
        if name_check.endswith("GC"):
            col_name = "MGC"
        elif name_check.endswith("downsampled"):
            col_name = "NGC"
        else:
            col_name = "AGC"
        tracking_data = pd.read_hdf(fl_pth)
        heading = 'DLC_resnet50_DLC'+ date +'shuffle1_1030000'
        necessary_points = tracking_data[heading][feature_names]
        
        lik = pd.DataFrame(necessary_points[i].likelihood)
        lik = lik.rename(columns={"likelihood" : col_name})
        df2 = pd.concat([df2, lik], axis = 1)
        mul_cols.append((i, col_name))

    df2.columns=pd.MultiIndex.from_tuples(mul_cols)
    conf = pd.concat([conf, df2], axis = 1)


In [11]:
# fl_name = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/Confidence_combined_4extracted.csv'

# # open the file in the write mode
# f = open(fl_name, 'w')

# # write a row to the csv file
# conf.to_csv(fl_name, index=False)

In [12]:
# fl_name = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/Confidence_combined_4extracted.csv'

# conf_quarts = pd.read_csv(fl_name, header=[0,1])

conf_quarts = conf

In [8]:
# # first quarter

# conf_quart1 = conf_quarts[conf_quarts.le(0.10).any(axis = 1)]

# means = []
# std_divs = []
# pdfs = []
# sort_conf = []

# for i in conf_quart1.columns:
#     mean_i = np.mean(conf_quart1[i])
#     std_div_i = np.mean(conf_quart1[i])
#     sort_i = conf_quart1[i].sort_values()
    
#     means.append(mean_i)
#     std_divs.append(std_div_i)
#     pdfs.append(stats.norm.pdf(sort_i, mean_i, std_div_i))
#     sort_conf.append(sort_i)


In [9]:
# cols = list(conf_quart1.columns)
# feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']
# colname = ['MGC','NGC','AGC']

In [10]:
# conf_quart1.boxplot(rot = 90 ,showfliers=False)

In [13]:
conf_diff = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    cdf = conf[i]
    mul_cols = []
    for j in range(3):
        cur_name = cdf.columns[j]
        prev_name = cdf.columns[j-1]
        col_name = cur_name[0]+prev_name[0]
        diff = pd.DataFrame((cdf[cur_name] - cdf[prev_name]).abs(), columns=[col_name])
        # diff = pd.DataFrame((cdf[cur_name] - cdf[prev_name]), columns=[col_name])
        df2 = pd.concat([df2, diff], axis = 1)
        mul_cols.append((i, col_name))
    df2.columns=pd.MultiIndex.from_tuples(mul_cols)
    conf_diff = pd.concat([conf_diff, df2], axis = 1)

In [14]:
filtx = conf_quarts.xs('NGC', level = 1, axis = 1)
filtx = filtx[filtx.le(0.2).all(axis = 1)]
filtg = conf_diff.loc[filtx.index]
filtb = filtg.xs('AN', level = 1, axis = 1)
filtb = filtb[filtb.gt(0.7).all(axis = 1)]

res = conf_quarts.loc[filtb.index]
pd.options.display.max_rows = 460
res

RightEye_Pupil                     LeftEye_Pupil                      \
                   MGC       NGC       AGC           MGC       NGC       AGC   
5154          0.794521  0.003434  0.986697      0.971676  0.110499  0.880737   
106045        0.999964  0.086167  0.999999      0.541265  0.008776  0.926258   
109237        0.467492  0.000976  0.966359      0.869575  0.003079  0.994931   
109238        0.985437  0.000968  0.997506      0.974929  0.003157  0.997247   
184395        0.975695  0.001417  0.999795      0.328957  0.000820  0.973129   
256092        0.021967  0.003409  0.976004      0.007556  0.003138  0.983550   
256098        0.003164  0.027989  0.997364      0.000271  0.002810  0.904214   
256260        0.000774  0.001451  0.998467      0.000264  0.003980  0.944656   
256282        0.067062  0.001655  0.818436      0.072354  0.004872  0.980552   
256283        0.074866  0.001659  0.775378      0.088994  0.004825  0.980349   
256303        0.207010  0.001509  0.955461      0.022160  0.004447  0.982990   
256326        0.421242  0.118249  0.983619      0.180572  0.075353  0.952970   
332743        0.998343  0.002327  0.997336      0.999192  0.005639  0.995351   
332744        0.998984  0.010751  0.999467      0.999990  0.012404  0.999913   
339463        0.999948  0.006430  0.999930      0.915001  0.059323  0.985851   
339464        0.999677  0.000911  0.999628      0.649057  0.000845  0.892583   
339465        0.999684  0.000881  0.999527      0.615523  0.000837  0.902733   
355989        0.970030  0.000704  0.986934      0.999464  0.000665  0.999552   
386628        0.765320  0.002135  0.881123      0.959768  0.002018  0.875269   
517123        0.999921  0.001037  0.999570      0.999245  0.004761  0.990819   
517124        0.999990  0.000996  0.999926      0.999979  0.000394  0.999912   
517125        0.999992  0.000972  0.999940      0.999984  0.000374  0.999934   
518130        0.999700  0.000752  0.998991      0.999321  0.000323  0.999715   
518131        0.999777  0.000803  0.999483      0.998689  0.000340  0.999571   
518132        0.999961  0.001018  0.999847      0.999915  0.020973  0.999771   
518133        0.999960  0.001180  0.999828      0.999918  0.021884  0.999724   
518134        0.999989  0.001227  0.999879      0.999940  0.020035  0.999918   
518138        0.999956  0.001346  0.999857      0.999901  0.000544  0.999956   
518139        0.999962  0.001362  0.999843      0.999926  0.000552  0.999961   
518148        0.999962  0.002832  0.999250      0.921012  0.002221  0.960262   
518149        0.999960  0.002670  0.999258      0.922398  0.002243  0.965583   
531206        0.999992  0.043871  0.999989      0.970707  0.022471  0.983891   
531207        0.999993  0.043530  0.999988      0.970379  0.022542  0.983775   
567178        0.974381  0.000862  0.904361      0.999998  0.000575  0.999983   
567511        0.497273  0.001255  0.885857      0.998941  0.007458  0.999559   
567513        0.682568  0.001033  0.739140      0.999334  0.000327  0.996949   
568154        0.853050  0.000609  0.985160      0.846958  0.007208  0.983729   
568155        0.938823  0.000528  0.986943      0.790245  0.005968  0.983832   
569572        0.994473  0.028454  0.999385      0.701656  0.003478  0.996014   
569573        0.990146  0.025820  0.999289      0.696609  0.003411  0.991911   
569574        0.000803  0.000838  0.929634      0.001398  0.000291  0.857022   
569575        0.000293  0.000860  0.946786      0.001705  0.000298  0.878018   
570169        0.998254  0.001252  0.976881      0.997912  0.147655  0.978151   
570170        0.996880  0.001253  0.990215      0.997707  0.146320  0.984302   
570171        0.970090  0.001190  0.987874      0.999684  0.132508  0.996725   
570172        0.927302  0.001189  0.993671      0.999680  0.145123  0.997528   
573780        0.999658  0.000630  0.999360      0.999989  0.175926  0.999978   
578403        0.999997  0.000783  0.999997      0.997877  0.000314  0.996217   
578404  

In [13]:
# import researchpy as rp
# from pprint import pprint

In [15]:
cutoff = 0.9
    
# filtx = conf_quarts.xs('NGC', level = 1, axis = 1)
# filtx = filtx[filtx.le(cutoff).all(axis = 1)]

print(f"Filtered for NGC and AGC < {cutoff}\n\n")

for i in feature_names:
    print(i)
    subset = conf_quarts[i]
    # pairs = {1: ['MGC', 'AGC'],
             # 2: ['NGC', 'MGC'],
             # 3: ['AGC', 'NGC']}
    # pairs = {1: ['MA'],
    #          2: ['NM'],
    #          3: ['AN']}
    # for j in range(3):
    #     print(pairs[j+1])
    #     # print(stats.wilcoxon(subset[pairs[j+1][0]], subset[pairs[j+1][1]], alternative="two-sided", nan_policy='omit', method='exact', zero_method='zsplit'))
    duo = pd.DataFrame()
    duo = pd.concat([conf_quarts[i]['NGC'], conf_quarts[i]['AGC'], conf_quarts[i]['MGC'], conf_diff[i]['AN'], conf_diff[i]['MA']], axis = 1)
    
    # duo = duo.loc[filtx.index]
    duo = duo[duo['NGC'].gt(cutoff)]
    duo = duo[duo['AGC'].gt(cutoff)]
    # duo = duo[duo['NGC'].le(0.9)]
    # duo = duo[duo['AGC'].gt(0.7)]
    
    
    print(duo.shape)
    print("Mean of NGC - ", np.mean(duo['NGC']))
    print("Std_div of NGC - ", np.std(duo['NGC']))
    print("Std_error of NGC - ", stats.sem(duo['NGC']))
    print()
    print("Mean of MGC - ", np.mean(duo['MGC']))
    print("Std_div of MGC - ", np.std(duo['MGC']))
    print("Std_error of MGC - ", stats.sem(duo['MGC']))
    print()
    print("Mean of AGC - ", np.mean(duo['AGC']))
    print("Std_div of AGC - ", np.std(duo['AGC']))
    print("Std_error of AGC - ", stats.sem(duo['AGC']))
    print()
    print(stats.pearsonr(duo['NGC'], duo['AN']), ' - NGC and AN')
    print(stats.wilcoxon(duo['NGC'], duo['AGC']), ' - NGC and AGC')
    print(stats.ttest_1samp(duo['AN'], popmean=0), ' - of AN\n')
    print(stats.pearsonr(duo['MGC'], duo['MA']), ' - MGC and MA')
    print(stats.wilcoxon(duo['MGC'], duo['AGC']), ' - MGC and AGC')
    print(stats.ttest_1samp(duo['MA'], popmean=0), ' - of MA\n')
    # print(stats.wilcoxon(duo['AN']), 'AN')
    print(stats.f_oneway(duo['NGC'], duo['AGC'], duo['MGC']), ' - of all three(NGC, MGC and AGC)\n\n\n')


Filtered for NGC and AGC < 0.9


RightEye_Pupil
(43293, 5)
Mean of NGC -  0.9948599781714806
Std_div of NGC -  0.014481834072615614
Std_error of NGC -  6.960168318111255e-05

Mean of MGC -  0.9915070253104528
Std_div of MGC -  0.07161180327653857
Std_error of MGC -  0.00034417616019415993

Mean of AGC -  0.9991308063329928
Std_div of AGC -  0.004276261541554851
Std_error of AGC -  2.0552300179829347e-05

PearsonRResult(statistic=-0.9751918157984673, pvalue=0.0)  - NGC and AN
WilcoxonResult(statistic=204066284.0, pvalue=0.0)  - NGC and AGC
Ttest_1sampResult(statistic=72.03733763734449, pvalue=0.0)  - of AN

PearsonRResult(statistic=-0.9989694557951209, pvalue=0.0)  - MGC and MA
WilcoxonResult(statistic=378609061.0, pvalue=2.291816887279593e-218)  - MGC and AGC
Ttest_1sampResult(statistic=23.89244373477697, pvalue=2.385894104343768e-125)  - of MA

F_onewayResult(statistic=354.0311038030301, pvalue=4.611560898110384e-154)  - of all three(NGC, MGC and AGC)



LeftEye_Pupil
(40705, 5)
Mean 

In [16]:
cutoff = 0.9
    
# filtx = conf_quarts.xs('NGC', level = 1, axis = 1)
# filtx = filtx[filtx.le(cutoff).all(axis = 1)]

print(f"Filtered for NGC and AGC < {cutoff}\n\n")

for i in feature_names:
    print(i)
    duo = pd.DataFrame()
    duo = pd.concat([conf_quarts[i]['NGC'], conf_quarts[i]['AGC'], conf_quarts[i]['MGC'], conf_diff[i]['AN'], conf_diff[i]['MA']], axis = 1)
    
    # duo = duo.loc[filtx.index]
    # duo = duo[duo['NGC'].gt(cutoff)]
    # duo = duo[duo['AGC'].gt(cutoff)]
    duo = duo[duo['NGC'].le(0.7)]
    duo = duo[duo['AGC'].gt(0.9)]
    
    
    print(duo.shape)
    # print("Mean of NGC - ", np.mean(duo['NGC']))
    # print("Std_div of NGC - ", np.std(duo['NGC']))
    # print("Std_error of NGC - ", stats.sem(duo['NGC']))
    # print()
    # print("Mean of MGC - ", np.mean(duo['MGC']))
    # print("Std_div of MGC - ", np.std(duo['MGC']))
    # print("Std_error of MGC - ", stats.sem(duo['MGC']))
    # print()
    # print("Mean of AGC - ", np.mean(duo['AGC']))
    # print("Std_div of AGC - ", np.std(duo['AGC']))
    # print("Std_error of AGC - ", stats.sem(duo['AGC']))
    # print()
    # print(stats.pearsonr(duo['NGC'], duo['AN']), ' - NGC and AN')
    # print(stats.wilcoxon(duo['NGC'], duo['AGC']), ' - NGC and AGC')
    # print(stats.ttest_1samp(duo['AN'], popmean=0), ' - of AN\n')
    # print(stats.pearsonr(duo['MGC'], duo['MA']), ' - MGC and MA')
    # print(stats.wilcoxon(duo['MGC'], duo['AGC']), ' - MGC and AGC')
    # print(stats.ttest_1samp(duo['MA'], popmean=0), ' - of MA\n')
    # # print(stats.wilcoxon(duo['AN']), 'AN')
    # print(stats.f_oneway(duo['NGC'], duo['AGC'], duo['MGC']), ' - of all three(NGC, MGC and AGC)\n\n\n')


Filtered for NGC and AGC < 0.9


RightEye_Pupil
(7891, 5)
LeftEye_Pupil
(6009, 5)
NostrilsTop_Centre
(4334, 5)
OutlineTop_Mid
(3907, 5)


In [18]:
filtx = conf_quarts.xs('NGC', level = 1, axis = 1)
filtx = filtx[filtx.gt(0.70).all(axis = 1)]


duo = conf_quarts
duo = duo.loc[filtx.index]
print(duo.shape)

filty = duo.xs('AGC', level = 1, axis = 1)
filty = filty[filty.gt(0.95).all(axis = 1)]
duo = duo.loc[filty.index]
print(filtx.shape, filty.shape)
print(duo.shape)
print(duo.index)


(37882, 12)
(37882, 4) (35082, 4)
(35082, 12)
Int64Index([    395,     396,     397,     398,     399,     400,     401,
                402,     408,     409,
            ...
            1869392, 1869393, 1869395, 1869397, 1869398, 1869403, 1869525,
            1869526, 1869542, 1869694],
           dtype='int64', length=35082)


In [22]:
def crop_it(frame_numbers):
    path = dict([(1,'/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/GC_Conv_vids/combined_vidsdownsampled_72_2_5_5_AGC_clahe.mp4'),
                (0,'/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/GC_Conv_vids/combined_vidsdownsampled.mp4')])
    video = cv2.VideoCapture(path[0])
    total_frames = len(frame_numbers)
    fps = 30
    
    today_1 = datetime.date.today()
    today = str(today_1)
    
    output_name = os.path.join(os.getcwd(), f'Frames_to_label_{today}.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
    frame_number = 0
    origin = []
    
    for j in range(len(frame_numbers)):
        i = frame_numbers[j]
        vid = random.randint(0,1)
        
        cap = cv2.VideoCapture(path[vid])
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            print('Error - frame not retreived')
            continue
        
        origin.append(vid)
        
        out.write(frame)
        cap.release()
        print(round(frame_number/total_frames*100, 2), end = '\r')
        frame_number += 1
        
    video.release()
    out.release()
    cv2.destroyAllWindows()
    print("Done")
    return output_name, origin
    # print('')

In [23]:
frames_needed = duo.index
file_path = crop_it(frames_needed)

Done0


In [41]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids'

In [48]:
fl_name = os.path.join(os.getcwd(), 'Frame_source_info.csv')
    
# open the file in the write mode
# f = open(fl_name, 'w')

# write a row to the csv file
# for i in file_path[1]:
#     f.write(f'{i}\n')
#     # print(i)

data = pd.DataFrame([frames_needed, file_path[1]]).T
data.to_csv(fl_name, index=False)
data

,0,1
0,395,1
1,396,1
2,397,0
3,398,1
4,399,0
...,...,...
35077,1869403,0
35078,1869525,0
35079,1869526,1
35080,1869542,1


In [33]:
import deeplabcut

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [30]:
project_name = 'DLC'
your_name = 'YR'
folder = ['/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/Frames_to_label_2023-12-03.mp4'] 

In [31]:
def process(fold, project_name, your_name):
    file = fold[0]
    print(file)
    bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
                 'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
    videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
    video_down = folder
    name_fold = 'Analyse_' + os.path.basename(os.path.dirname(file))
    
    model_options = deeplabcut.create_project.modelzoo.Modeloptions
    model_selection = 'primate_face'

    config_path, train_config_path = deeplabcut.create_pretrained_project(
        project_name,
        your_name,
        video_down,
        videotype=videotype,
        model=model_selection,
        analyzevideo=True,
        createlabeledvideo=False,
        copy_videos=False,
    )

    edits = {
        'dotsize': 1.2,  # size of the dots!
        'pcutoff': 0.4,  # the higher, the more conservative the plotting!
    }
    deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)
    project_path = os.path.dirname(config_path)
    full_video_path = []
    for i in fold:
        full_video_path.append(os.path.join(project_path,'videos', os.path.basename(i)))

    # filter predictions (should already be done above ;):
    deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

    # re-create the video with your edits!
    # deeplabcut.CropVideo(config_path, full_video_path, 'crop', 
    deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)
    
    

In [32]:
process(folder, project_name, your_name)
# notify("DLC labelling done")

/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/Frames_to_label_2023-12-03.mp4


NameError: name 'deeplabcut' is not defined

In [101]:
config_path = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-12-01/config.yaml'

In [102]:
deeplabcut.extract_frames(config_path, mode='automatic', algo='uniform)

ImportError: cannot import name 'frame_extraction_toolbox' from 'deeplabcut.gui' (/home/yramakrishna/anaconda3/lib/python3.9/site-packages/deeplabcut/gui/__init__.py)

In [ ]:
deeplabcut.label_frames(config_path)

  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]
  Plugin uses incompatible Qt library (5.15.0) [release]



In [79]:
# Extracts confidence of each point in each video, grouped under points.

feature_names = ['RightEye_Pupil','LeftEye_Pupil']#, 'RightEye_Inner', 'LeftEye_Inner']
# feature_names = ['RightEye_Pupil', 'LeftEye_Pupil', 'OutlineTop_Mid', 'NostrilsTop_Centre', 'RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']


conf = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    mul_cols = []
    for j in c_s:
        fl_pth, vid_pth, x, date = j
        name_check = os.path.basename(vid_pth).split('DLC')[0]
        if name_check.endswith("GC"):
            col_name = "MGC"
        elif name_check.endswith("downsampled"):
            col_name = "NGC"
        else:
            col_name = "AGC"
        tracking_data = pd.read_hdf(fl_pth)
        heading = 'DLC_resnet50_DLC'+ date +'shuffle1_1030000'
        necessary_points = tracking_data[heading][feature_names]
        
        print(necessary_points)
        
        # lik = pd.DataFrame(necessary_points[i])
        # lik = lik.rename(columns={"likelihood" : col_name})
#         df2 = pd.concat([df2, lik], axis = 1)
#         mul_cols.append((i, col_name))

#     # df2.columns=pd.MultiIndex.from_tuples(mul_cols)
#     conf = pd.concat([conf, df2], axis = 1)


TypeError: unhashable type: 'Index'

In [77]:
conf

coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
0,315.097260,0.823932,0.000143,315.274567,0.998680,0.000397,1.934032,1.008060,0.000137,315.023773,2.837302,0.000113,199.291473,0.000000,0.000208,1.729270,2.396275,0.000056
1,315.104279,0.920947,0.000133,315.274567,0.999196,0.000508,315.987579,1.008060,0.000143,315.037231,2.904608,0.000111,199.292191,0.000000,0.000289,313.962799,2.396275,0.000064
2,315.133453,0.942880,0.000132,315.286713,1.015165,0.000508,315.987579,1.059260,0.000145,315.158112,2.917567,0.000110,314.190430,2.275521,0.000290,313.962799,2.396275,0.000065
3,315.133453,0.972158,0.000123,315.286865,1.015165,0.000422,315.987579,1.059260,0.000141,315.140320,2.992493,0.000101,314.190430,2.275521,0.000217,313.962799,2.379015,0.000066
4,315.177216,1.008314,0.000105,315.287903,1.022236,0.000408,315.917358,1.034693,0.000158,315.158112,3.000335,0.000091,314.233398,2.324283,0.000212,313.943451,2.365713,0.000073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869914,NaN,NaN,NaN,316.255859,3.061479,0.000060,315.856354,3.030172,0.000071,NaN,NaN,NaN,314.345764,2.626508,0.000087,30.669464,-2.898792,0.000078
1869915,NaN,NaN,NaN,316.261261,3.062100,0.000053,315.831207,3.027593,0.000073,NaN,NaN,NaN,314.342346,2.625190,0.000080,30.675947,-2.871759,0.000080
1869916,NaN,NaN,NaN,316.261505,3.062100,0.000052,315.799133,3.019557,0.000072,NaN,NaN,NaN,314.342346,2.624088,0.000079,30.675947,-2.869036,0.000078
1869917,NaN,NaN,NaN,316.261261,3.062100,0.000053,315.771393,3.005276,0.000071,NaN,NaN,NaN,314.340393,2.624088,0.000080,30.675947,-2.865801,0.000079


In [ ]:
# # conf_quart1 = conf_quarts[conf_quarts.le(0.25).all(axis = 1)]
# conf_quart2 = conf_quarts*100000
# # for j in range(4):
#     # print((j+1)/4)
# for i in feature_names:
#     print(i)
#     subset = conf_quarts[i]
#     # pairs = {1: ['MGC', 'AGC'],
#              # 2: ['NGC', 'MGC'],
#              # 3: ['AGC', 'NGC']}
#     # pairs = {1: ['MA'],
#     #          2: ['NM'],
#     #          3: ['AN']}
#     # for j in range(3):
#     #     print(pairs[j+1])
#     #     # print(stats.wilcoxon(subset[pairs[j+1][0]], subset[pairs[j+1][1]], alternative="two-sided", nan_policy='omit', method='exact', zero_method='zsplit'))
#     duo = pd.DataFrame()
#     duo = pd.concat([conf_quarts[i]['NGC'], conf_quarts[i]['AGC'], conf_quarts[i]['MGC'], conf_diff[i]['AN']], axis = 1)
#     duo = duo[duo['NGC'].gt(0.01)]
#     # [ print(duo)
#     print(stats.pearsonr(duo['NGC'], duo['AN']))
#     print(stats.wilcoxon(duo['NGC'], duo['AGC']))
#     print(stats.f_oneway(duo['NGC'], duo['AGC'], duo['MGC']))
#     # pprint(stats.f_oneway(duo['NGC'], duo['AGC'], duo['MGC']))
#     # print(rp.summary_cont(duo['NGC'], duo['AGC'], duo['MGC']))
#     print()

In [ ]:
# import scipy.stats
# t_stat, p_value = ttest_ind(conf_quart1[cols[1]], conf_quart1[cols[2]])
# print("T-statistic value: ", t_stat)  
# print("P-Value: ", p_value)

In [ ]:
# # Drawing a graph
# plt.plot(sort_conf[0], pdfs[0])
# plt.xlim([-0.01,0.01])  
# plt.xlabel("Grain weight (mg)", size=12)    
# plt.ylabel("Frequency", size=12)                
# plt.grid(True, alpha=0.3, linestyle="--")
# plt.show()